<a href = "https://www.pieriantraining.com"><img src="../PT Centered Purple.png"> </a>

<em style="text-align:center">Copyrighted by Pierian Training</em>

# Understanding Prompt Templates

In [2]:
import os
api_key = os.environ["OPENAI_API_KEY"]

In [5]:
from langchain.llms import OpenAI
llm = OpenAI(openai_api_key=api_key)
print(llm('Here is a fun fact about Pluto:'))

 

Pluto was reclassified as a dwarf planet in 2006, making it the largest object in the Kuiper belt.


You can also use generate to get full output with more info:

In [6]:
# NEEDS TO BE A LIST, EVEN FOR JUST ONE STRING
result = llm.generate(['Here is a fun fact about Pluto:',
                     'Here is a fun fact about Mars:']
                     )

In [7]:
result.schema()

{'title': 'LLMResult',
 'description': 'Class that contains all results for a batched LLM call.',
 'type': 'object',
 'properties': {'generations': {'title': 'Generations',
   'type': 'array',
   'items': {'type': 'array', 'items': {'$ref': '#/definitions/Generation'}}},
  'llm_output': {'title': 'Llm Output', 'type': 'object'},
  'run': {'title': 'Run',
   'type': 'array',
   'items': {'$ref': '#/definitions/RunInfo'}}},
 'required': ['generations'],
 'definitions': {'Generation': {'title': 'Generation',
   'description': 'A single text generation output.',
   'type': 'object',
   'properties': {'text': {'title': 'Text', 'type': 'string'},
    'generation_info': {'title': 'Generation Info', 'type': 'object'},
    'type': {'title': 'Type',
     'default': 'Generation',
     'enum': ['Generation'],
     'type': 'string'}},
   'required': ['text']},
  'RunInfo': {'title': 'RunInfo',
   'description': 'Class that contains metadata for a single execution of a Chain or model.',
   'type': '

In [8]:
result

LLMResult(generations=[[Generation(text='\n\nPluto is the only dwarf planet in our Solar System that has a moon larger than itself. Its largest moon, Charon, is over half the size of Pluto.', generation_info={'finish_reason': 'stop', 'logprobs': None})], [Generation(text='\n\nMars has the tallest mountain of any planet in our solar system. The mountain, called Olympus Mons, is 21 kilometers (13 miles) tall and about 600 kilometers (370 miles) in diameter.', generation_info={'finish_reason': 'stop', 'logprobs': None})]], llm_output={'token_usage': {'total_tokens': 93, 'prompt_tokens': 16, 'completion_tokens': 77}, 'model_name': 'text-davinci-003'}, run=[RunInfo(run_id=UUID('dbc7e940-9937-4f6f-81ae-e9d4987fd66f')), RunInfo(run_id=UUID('70bcf323-cd28-4363-915d-6ffc5246e457'))])

## Language Model Templates

### No Input Variables

In [9]:
from langchain import PromptTemplate

# An example prompt with no input variables
no_input_prompt = PromptTemplate(input_variables=[], template="Tell me a fact")
no_input_prompt.format()
# -> "Tell me a fact."

'Tell me a fact'

### Single Input Variable

In [10]:
# An example prompt with one input variable
one_input_prompt = PromptTemplate(input_variables=["topic"], template="Tell me a fact about {topic}.")
# Notice how the stirng "topic" gets automatically converted to a parameter name, very convienent! 
one_input_prompt.format(topic="Mars")
# -> "Tell me a fact about Mars"

'Tell me a fact about Mars.'

### Multiple Input Variables

In [27]:
# An example prompt with multiple input variables
multiple_input_prompt = PromptTemplate(
    input_variables=["topic", "level"], 
    template="Tell me a fact about {topic} for a student {level} level."
)
multiple_input_prompt.format(topic='Mars',level='8th Grade')

'Tell me a fact about Mars for a student 8th Grade level.'

# Chat Model Templates

Chat models require a list of chat messages called a prompt, which is different from a raw string that you would input into a language model. Each message in the prompt is associated with a role, such as AI, human, or system.

For instance, when using the OpenAI Chat Completion API, a chat message can be assigned the role of AI, human, or system. The model is designed to pay closer attention to instructions provided in system chat messages.

To simplify the process of constructing and working with prompts, LangChain offers various prompt templates. It is highly recommended to utilize these chat-related prompt templates instead of PromptTemplate when interacting with chat models. This will allow you to fully harness the potential of the underlying chat model and enhance your experience.

We will favor these models in the course due to upcoming changes in the OpenAI ecosystem where chat agents will be favored over text completion models.

In [12]:
from langchain.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    AIMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

In [13]:
system_template="You are an {language} coding expert great at writing recursive functions for everything"
system_message_prompt = SystemMessagePromptTemplate.from_template(system_template)

In [14]:
system_message_prompt.input_variables

['language']

In [15]:
human_template="{problem_statement}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)

In [16]:
human_message_prompt.input_variables

['problem_statement']

In [17]:
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

In [18]:
chat_prompt.input_variables

['language', 'problem_statement']

In [19]:
# get a chat completion from the formatted messages
chat_prompt.format_prompt(language="python", problem_statement="count number of nodes in binary tree").to_messages()

[SystemMessage(content='You are an python coding expert great at writing recursive functions for everything'),
 HumanMessage(content='count number of nodes in binary tree')]

In [23]:
request = chat_prompt.format_prompt(language="python", problem_statement="count number of nodes in binary tree")

In [24]:
print (type(request))

<class 'langchain_core.prompt_values.ChatPromptValue'>


In [25]:
print (request.to_string())

System: You are an python coding expert great at writing recursive functions for everything
Human: count number of nodes in binary tree


In [26]:
print (request.to_messages())

[SystemMessage(content='You are an python coding expert great at writing recursive functions for everything'), HumanMessage(content='count number of nodes in binary tree')]


## Prompt Templates with an LLM Call

In [27]:
from langchain.chat_models import ChatOpenAI
chat = ChatOpenAI(openai_api_key=api_key)

In [28]:
result = chat(request.to_messages())

In [31]:
print (result)

content='To count the number of nodes in a binary tree, we can use a recursive function that traverses through the tree and increments a counter for each node encountered.\n\nHere\'s an example implementation:\n\n```python\nclass Node:\n    def __init__(self, value):\n        self.value = value\n        self.left = None\n        self.right = None\n\ndef count_nodes(root):\n    if root is None:\n        return 0\n\n    return 1 + count_nodes(root.left) + count_nodes(root.right)\n```\n\nIn this implementation, the `count_nodes` function takes the root node of the binary tree as input and returns the total number of nodes in the tree.\n\nWe start by checking if the root node is None. If it is, then the tree is empty and we return 0.\n\nIf the root node is not None, we increment the counter by 1 and recursively call the `count_nodes` function on both the left and right subtrees. The sum of the counts from the left and right subtrees, along with the root node, gives us the total number of n

In [32]:
print(result.content)

To count the number of nodes in a binary tree, we can use a recursive function that traverses through the tree and increments a counter for each node encountered.

Here's an example implementation:

```python
class Node:
    def __init__(self, value):
        self.value = value
        self.left = None
        self.right = None

def count_nodes(root):
    if root is None:
        return 0

    return 1 + count_nodes(root.left) + count_nodes(root.right)
```

In this implementation, the `count_nodes` function takes the root node of the binary tree as input and returns the total number of nodes in the tree.

We start by checking if the root node is None. If it is, then the tree is empty and we return 0.

If the root node is not None, we increment the counter by 1 and recursively call the `count_nodes` function on both the left and right subtrees. The sum of the counts from the left and right subtrees, along with the root node, gives us the total number of nodes in the binary tree.

Here's